In [1]:
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import copy
import re

import math
import torch

import json
from collections import defaultdict
import dill
import random

import torch.nn.functional as F
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.nn.utils.rnn import pack_padded_sequence

from tqdm.notebook import tqdm
import pickle as pkl

from torch.nn.utils.rnn import pad_packed_sequence


In [2]:
from rule_based_utils import *
from chordify_json_extension import all_chords, check_chordify, gram2chordify_chord, convert2chordify
from thresholding_and_grouping import *
from rule_based import give_subset, get_uni_gram, train_all, RuleModel, evaluate_test_set

In [3]:
cols2extract = ['song','CFP','CFD',
                'UC','RHC','CPT',
                'BD','R','weighted_total']


files_and_annotations = extract_chord_path_and_annotations("Annotations.csv", cols2extract, delimiter=",")
chords_and_annotations = get_chords_and_annotations(files_and_annotations, no_duplicates=True)

## get all uni, bi, tri, and quad grams

In [6]:
all_uni_gram, all_bi_gram, all_tri_gram, all_quad_gram = {},{},{},{}
for (chords, _, sub_scores, total) in chords_and_annotations:
    long_string = process_song(chords,  remove_nl=True)
    if long_string.find('||') != -1:
        print(chords, '\n\n', long_string)
        break
    new_uni, new_bi, new_tri, new_quad = extract_n_grams(long_string)
    all_uni_gram = update_dict(new_uni, all_uni_gram)
    all_bi_gram = update_dict(new_bi, all_bi_gram)
    all_tri_gram = update_dict(new_tri, all_tri_gram)
    all_quad_gram = update_dict(new_quad, all_quad_gram)


In [8]:
stripped_uni_gram_keys= simplify_grams(all_uni_gram)
stripped_bi_gram_keys= simplify_grams(all_bi_gram)
stripped_tri_gram_keys= simplify_grams(all_tri_gram)        
stripped_quad_gram_keys= simplify_grams(all_quad_gram)


In [9]:
print("all grams without simplification:")
print(len(all_uni_gram),len(all_bi_gram),len(all_tri_gram),len(all_quad_gram),)
print("all grams simplified (removing all () and / )")
print(len(stripped_uni_gram_keys),len(stripped_bi_gram_keys),len(stripped_tri_gram_keys),len(stripped_quad_gram_keys),)


all grams without simplification:
545 2722 4992 6842
all grams simplified (removing all () and / )
226 1946 4187 6175


### Compute IDF dicts for original and simplified grams

In [11]:
all_idf_dict = get_idf_dict(chords_and_annotations,n_gram=1,simplified=False)
all_idf_dict = {k:math.log(200/v) for k,v in all_idf_dict.items()}

simplified_idf_dict = get_idf_dict(chords_and_annotations,n_gram=1,simplified=True)
simplified_idf_dict = {k:math.log(200/v) for k,v in simplified_idf_dict.items()}


### score uncommonness of chords in three boundary threshold finding ways

In [12]:
assert len(all_chords.keys()) == 3721, "somethign wrong should have 3721 keys"

In [16]:
# check in uni_gram chords are in chordify guitargram dict
for keys in all_uni_gram.keys():
    if keys[0].find('/') != -1 and keys[0].find('(') == -1:
        conv, simpl = convert2chordify(keys[0], simplify=True)
        if not check_chordify(conv, print_all=False):
            print(f"not in chordify: {keys[0]} converted to: {conv}")
        if simpl:
            print(f"simplified: '{keys[0]}' to '{conv}'")
#         print(check_chordify(conv, print_all=False), simpl)

simplified: 'B:aug/5' to 'B:aug'
simplified: 'D:min9/5' to 'D:min9'
simplified: 'Cb:maj/5' to 'Cb:maj'
simplified: 'Eb:min/b7' to 'Eb:min'
simplified: 'Gb:maj7/2' to 'Gb:maj7'
simplified: 'F:9/5' to 'F:9'
simplified: 'A:5/6' to 'A:5'


## Generate 10 random splits for the k-fold

In [ ]:
s= list(range(200))

In [ ]:
random.shuffle(s)
split_size = 20
split_1_ind = s[-split_size:]
del s[-split_size:]
split_2_ind = s[-split_size:]
del s[-split_size:]
split_3_ind = s[-split_size:]
del s[-split_size:]
split_4_ind = s[-split_size:]
del s[-split_size:]
split_5_ind = s[-split_size:]
del s[-split_size:]
split_6_ind = s[-split_size:]
del s[-split_size:]
split_7_ind = s[-split_size:]
del s[-split_size:]
split_8_ind = s[-split_size:]
del s[-split_size:]
split_9_ind = s[-split_size:]
del s[-split_size:]
split_10_ind = s[-split_size:]
del s[-split_size:]


In [ ]:
print(split_1_ind)
print(split_2_ind)
print(split_3_ind)
print(split_4_ind)
print(split_5_ind)
print(split_6_ind)
print(split_7_ind)
print(split_8_ind)
print(split_9_ind)
print(split_10_ind)


In [ ]:
## final indices
split_1_ind =[88, 71, 4, 171, 33, 56, 105, 72, 139, 64, 26, 117, 151, 183, 80, 153, 121, 63, 176, 97]
split_2_ind =[132, 182, 3, 168, 39, 165, 147, 109, 114, 0, 177, 133, 160, 198, 20, 75, 32, 123, 157, 193]
split_3_ind =[167, 57, 100, 195, 27, 118, 128, 190, 103, 141, 162, 186, 13, 81, 55, 8, 36, 135, 50, 2]
split_4_ind =[172, 79, 10, 148, 73, 92, 164, 34, 40, 47, 11, 25, 98, 85, 136, 5, 142, 99, 161, 126]
split_5_ind =[102, 62, 23, 74, 178, 170, 112, 24, 106, 95, 152, 90, 101, 138, 21, 197, 7, 150, 77, 42]
split_6_ind =[35, 187, 52, 194, 115, 16, 116, 6, 53, 45, 15, 67, 131, 185, 191, 1, 96, 28, 19, 66]
split_7_ind =[83, 9, 59, 175, 76, 22, 51, 69, 14, 94, 38, 46, 154, 91, 104, 173, 93, 108, 48, 12]
split_8_ind =[180, 86, 137, 119, 43, 120, 84, 49, 58, 18, 89, 179, 155, 166, 111, 129, 192, 143, 134, 124]
split_9_ind =[17, 159, 82, 61, 41, 54, 184, 130, 199, 188, 181, 78, 60, 169, 174, 37, 29, 140, 145, 163]
split_10_ind =[144, 87, 70, 127, 68, 44, 196, 158, 31, 125, 113, 146, 122, 65, 156, 30, 149, 189, 107, 110]
split_dict = {
    1: split_1_ind,
    2: split_2_ind,
    3: split_3_ind,
    4: split_4_ind,
    5: split_5_ind,
    6: split_6_ind,
    7: split_7_ind,
    8: split_8_ind,
    9: split_9_ind,
    0: split_10_ind,
}

### Do k-fold cross validation for rule based

In [ ]:
save_performance = []
for i in range(10):
    # obtain train-validate-test-splits
    indices = np.asarray(list(range(10))) + i
    
    train_ind = [split_dict[(m% 10)] for m in indices[:-1]]
    validate_ind = [split_dict[(m % 10)] for m in indices[-1:]]

    train_ind = [k for j in train_ind for k in j]
    validate_ind = [k for j in validate_ind for k in j]
    
    # obtain chords, lab_files and rest of data
    train = give_subset(train_ind, chords_and_annotations)
    validate = give_subset(validate_ind, chords_and_annotations)
    
    # obtain idf and gram dicts
    train_idf_dict = get_idf_dict(train, n_gram=1, simplified=False)
    train_idf_dict = {k:math.log(200/v) for k,v in train_idf_dict.items()}
    train_idf_dict = defaultdict(lambda:max(train_idf_dict.values()),train_idf_dict)
    train_uni_grams = get_uni_gram(train)
    
    # "train" rule based model
    cfg = train_all(train, train_idf_dict, train_uni_grams)
    
    # create trained model
    kfold_model = RuleModel(cfg, train_uni_grams,  train_idf_dict)
    
    # evaluate for 
    all_performance = evaluate_test_set(kfold_model, validate, classify=False, custom_loss=True)
    save_performance.append(all_performance)

## Do k-fold cross validation for length-based

In [ ]:
def test_len(chords_and_annos, only_len=False, type_of_len='char'):
    cat_scores = {
        1: {1: [], 2: [], 3: [], 4: []},
        2: {1: [], 2: [], 3: [], 4: []},
        3: {1: [], 2: [], 3: [], 4: []},
        4: {1: [], 2: [], 3: [], 4: []},
        5: {1: [], 2: [], 3: [], 4: []},
        6: {1: [], 2: [], 3: [], 4: []},
        7: {1: [], 2: [], 3: [], 4: []},
    }
    weigthed_uni_gram_per_song = []

    lens = []
    
    for (chords, lab_file, sub_scores, total) in chords_and_annos:
        total_chord_count, song_score = 0, 0
        
        long_string = process_song(chords)
        long_string_nl = process_song(chords, remove_nl=False)
        annotations = long_string_nl.replace('\n', ' \n ').replace('  ', ' ').replace('||', '|')
        if type_of_len == 'char':
            cat_scores = len(annotations)
        elif type_of_len == 'dotsplit':
            new_anno = []
            for cho in annotations.split(' '):
                if cho.find(':') != -1:
                    new_anno.append(cho.split(':')[0])
                    new_anno.append(cho.split(':')[1])
                else:
                    new_anno.append(cho)
            cat_scores = len(new_anno)
        elif type_of_len == 'guitar':
            annotations = annotations.split(' ')
            try:
                annotations.remove('')
            except:
                pass
            new_anno = ''
            for ann in annotations:
                if ann in ['|', '\n', 'N', '*']:
                    new_anno += ann + ' '
                else:
                    converted = convert2chordify(ann)
                    to_add = all_chords[converted[0]]['guitar']
                    new_anno += str(to_add)
            cat_scores= len(new_anno)

        if only_len:
            lens.append(cat_scores)

        for i in range(1, 8):
            cat_scores[i][sub_scores[i-1]].append(cat_scores)  
    
    if only_len:
        return lens
    return cat_scores

### extracting encoding dict for : split chords

In [ ]:
possible_chars = []
give_subset()
for chords, _, _, _ in chords_and_annotations:
    big_string = new_big_string(chords, remove_nl=False)
    for cho in big_string.split(' '):
        if cho.find(':') != -1:
            possible_chars.append(cho.split(':')[0])
            possible_chars.append(cho.split(':')[1])
        else:
            possible_chars.append(cho)

In [ ]:
char_dict = {k:i+2 for i,k in enumerate(set(possible_chars))}

In [ ]:
char_dict['unk'] = 1

### create new snippets

In [ ]:
%autoreload 0

In [ ]:
!pip install dill


In [ ]:
def run_stuff():
    sub_chords = []
    for i in range(10):
        # obtain train-validate-test-splits
        indices = np.asarray(list(range(10))) + i
        train_ind = [split_dict[(m% 10)] for m in indices[:-2]]
    #     validate_ind = [split_dict[(m % 10)] for m in indices[-4:-2]]
        test_ind = [split_dict[(m % 10)] for m in indices[-2:]]

        train_ind = [k for j in train_ind for k in j]
    #     validate_ind = [k for j in validate_ind for k in j]
        test_ind = [k for j in test_ind for k in j]

        # obtain chords, lab_files and rest of data
        train = give_subset(train_ind, chords_and_annotations)
    #     validate = give_subset(validate_ind, chords_and_annotations)
        test = give_subset(test_ind, chords_and_annotations)

        # obtain idf and gram dicts
        train_idf_dict = get_idf_dict(train, n_gram=1, simplified=False)
        train_idf_dict = {k:math.log(200/v) for k,v in train_idf_dict.items()}

        train_idf_dict = defaultdict(lambda:max(train_idf_dict.values()),train_idf_dict)
        train_uni_grams = get_uni_gram(train)

        # "train" rule based model
        cfg = train_all(train, train_idf_dict, train_uni_grams)

        # create trained model
        kfold_model = RuleModel(cfg, train_uni_grams,  train_idf_dict)
        with open(f'./rule_models/fold_{i}.pkl','wb') as f:
            dill.dump(kfold_model, f)
        with open(f'./rule_models/idf_dict_{i}.pkl','wb') as f:
            dill.dump(kfold_model.idf_dict, f)


In [ ]:
run_stuff()

In [ ]:
x = []

In [ ]:
if x:
    print("hello")

## Classify custom data

In [ ]:
custom_data = pd.read_csv("chords_with_transpose_and_difficulties.csv", delimiter='\t')
file_with_added_scores = "chords_with_transpose_and_difficulties_and_playability.csv"

In [ ]:
letter2num = {
    'B': 0,
    'C': 0,
    'C#': 1,
    'Db': 1,
    'D': 2,
    'D#': 3,
    'Eb': 3,
    'E': 4,
    'E#': 5,
    'Fb': 4,
    'F': 5,
    'F#': 6,
    'Gb': 6,
    'G': 7,
    'G#': 8,
    'Ab': 8,
    'A': 9,
    'A#': 10,
    'Bb': 10,
    'B': 11,
    'Cb': 11,
}
num2letter = {
    0 : 'C',
    1 : 'C#',
    2 : 'D',
    3 : 'Eb',
    4 : 'E',
    5 : 'F',
    6 : 'F#',
    7 : 'G',
    8 : 'Ab',
    9 : 'A',
    10 : 'Bb',
    11 : 'B',
}
def magic(chord, transposition):
    if chord == 'N':
        return chord
    root, rest = chord.split(':')
    root_number = letter2num[root]
    root_transposed = (root_number + transposition) % 12
    root_transposed = num2letter[root_transposed]
    return root_transposed + ':' + rest

In [ ]:
def extract_lab_seq(chordify_data, transposition):
    start = True
    phrase_counter = 0
    cleaned_sequence = ''
    lab_sequence = ''
    prev = 0
    cur = 0
    for chord in chordify_data:
        chord_info = chord.split(';')
        prev = cur
        cur = int(chord_info[0])
        if start:
            start = False
            cleaned_sequence += f'{chord_info[2]}\t| '
        if cur < prev:
            if phrase_counter >= 4:
                cleaned_sequence += f'|\nPHRASE{chord_info[2]}\t'
                phrase_counter = 0
            cleaned_sequence += '| '
            phrase_counter += 1
                

        cleaned_sequence += magic(chord_info[1],transposition) + ' '

        lab_sequence += chord_info[2] + '\t' + chord_info[3] + '\t' + chord_info[1] +'\n'
    cleaned_sequence += '|'
    return cleaned_sequence, lab_sequence
    
def rule_based_custom_data(seq_data, lab_data):
    models = []
    all_songs_classified = []
    for i in range(10):
        with open(f'./rule_models/fold_{i}.pkl', 'rb') as f:
            loaded_model = dill.load(f)
        with open(f'./rule_models/idf_dict_{i}.pkl','rb') as f:
            load_model_cat_dict = dill.load(f)
        loaded_model.idf_dict = load_model_cat_dict
        models.append(loaded_model)
                
    for chord, lab in zip(seq_data,lab_data):
        song_stats = []
        for model in models:
            song_stats.append(model.predict(chord, lab))
#             print(model.predict(chord, lab))
        all_songs_classified.append(song_stats)
#         print()
    return all_songs_classified


In [ ]:
only_seq = []
only_lab = []
for i, row in custom_data[["chords", "transposed_amount"]].iterrows():
    better_chords = row["chords"].split('\\n')
    better_chords.remove('')
    x = extract_lab_seq(better_chords, row["transposed_amount"])
    only_seq.append(x[0].split('PHRASE'))
    only_lab.append(x[1])

## generate lstm playability

In [ ]:
from data import retrieve
from models import LSTMBase
from rule_based_utils import new_big_string
data = retrieve('billboard_salami')
custom = retrieve('custom')


In [ ]:
# only to extract encoding dict for trained models
for p in ['CFP']:#,'CFD','UC','RHC','CPT','BD','R','weighted_total']:
    all_data = data(
        seq_len=-1,#config['seq_len'],
        batch_size=2,#config['batch_size'],
        num_workers=0,#config['num_workers'],
        encoding='char',#config['encoding'],
        target=p,
        colla=True, #config['colla'],
        target_data_path='./Annotations_with_locations.csv'

    )
    all_data.custom_setup(1)


In [ ]:
custom_loader = custom(only_seq2_model_input, encode_dict = all_data.char_dict, encode_dict_size= all_data.char_dict_size)
train_loader = custom_loader.loader(batch_size=5)

## add rule-based scores

In [ ]:
all_the_outputs = rule_based_custom_data(only_seq, only_lab)

In [ ]:
all_all_1, all_all_2, all_all_3, all_all_4, all_all_5, all_all_6, all_all_7, all_all  = [],[],[],[],[],[],[],[]
cur_all_1 ,cur_all_2 ,cur_all_3 ,cur_all_4 ,cur_all_5 ,cur_all_6 ,cur_all_7 , cur_all = [],[],[],[],[],[],[],[]
for index, songies in enumerate(all_the_outputs):
    if index % 12 == 0:
        print(cur_all)#, np.argsort(cur_all))
        all_all_1.append(cur_all_1)
        all_all_2.append(cur_all_2)
        all_all_3.append(cur_all_3)
        all_all_4.append(cur_all_4)
        all_all_5.append(cur_all_5)
        all_all_6.append(cur_all_6)
        all_all_7.append(cur_all_7)
        all_all.append(cur_all)
        cur_all_1 ,cur_all_2 ,cur_all_3 ,cur_all_4 ,cur_all_5 ,cur_all_6 ,cur_all_7 , cur_all = [],[],[],[],[],[],[],[]
#         print()
    avg = [sum(y) / len(y) for y in zip(*songies)]
    cur_all_1.append(round(avg[0]/3, 4))
    cur_all_2.append(round(avg[1]/3, 4))
    cur_all_3.append(round(avg[2]/3, 4))
    cur_all_4.append(round(avg[3]/3, 4))
    cur_all_5.append(round(avg[4]/3, 4))
    cur_all_6.append(round(avg[5]/3, 4))
    cur_all_7.append(round(avg[6]/3, 4))
    cur_all.append(round(avg[7]/40, 4))

all_all_1.append(cur_all_1)
all_all_2.append(cur_all_2)
all_all_3.append(cur_all_3)
all_all_4.append(cur_all_4)
all_all_5.append(cur_all_5)
all_all_6.append(cur_all_6)
all_all_7.append(cur_all_7)
all_all.append(cur_all)


In [ ]:
len(unzipped_all_al)

In [ ]:
unzipped_all_al_1 = [y for x in all_all_1 for y in x]
unzipped_all_al_2 = [y for x in all_all_2 for y in x]
unzipped_all_al_3 = [y for x in all_all_3 for y in x]
unzipped_all_al_4 = [y for x in all_all_4 for y in x]
unzipped_all_al_5 = [y for x in all_all_5 for y in x]
unzipped_all_al_6 = [y for x in all_all_6 for y in x]
unzipped_all_al_7 = [y for x in all_all_7 for y in x] 
unzipped_all_al = [y for x in all_all for y in x]


In [ ]:
custom_data_rule_based['RB_CFP'] = unzipped_all_al_1
custom_data_rule_based['RB_CFD'] = unzipped_all_al_2
custom_data_rule_based['RB_UC'] = unzipped_all_al_3
custom_data_rule_based['RB_RHC'] = unzipped_all_al_4
custom_data_rule_based['RB_CPT'] = unzipped_all_al_5
custom_data_rule_based['RB_BD'] = unzipped_all_al_6
custom_data_rule_based['RB_R'] = unzipped_all_al_7
custom_data_rule_based['RB_weighted_total'] = unzipped_all_al


In [ ]:
custom_data

In [ ]:
for col in added_cols:
    custom_data[col] = all_thingies[col]

In [ ]:
custom_data.to_csv(file_with_added_scores)

## adding ML playability scores

In [ ]:
for i, term in enumerate(['CFP','CFD','UC','RHC','CPT','BD','R','weighted_total']):
    custom_data_rule_based[term] = ml_predicted_cat_playability[i]

In [ ]:
with open("all_scores_nrows_10000.pkl", 'rb') as f:
    playability0 = pkl.load(f)
with open("all_scores_10000_20000.pkl", 'rb') as f:
    playability1 = pkl.load(f)
with open("all_scores_20000_30000.pkl", 'rb') as f:
    playability2 = pkl.load(f)
with open("all_scores_30000_40000.pkl", 'rb') as f:
    playability3 = pkl.load(f)
with open("all_scores_40000_50000.pkl", 'rb') as f:
    playability4 = pkl.load(f)
with open("all_scores_50000_60000.pkl", 'rb') as f:
    playability5 = pkl.load(f)
with open("all_scores_60000_70000.pkl", 'rb') as f:
    playability6 = pkl.load(f)
with open("all_scores_70000_80000.pkl", 'rb') as f:
    playability7 = pkl.load(f)
with open("all_scores_80000_90000.pkl", 'rb') as f:
    playability8 = pkl.load(f)
with open("all_scores_90000_100000.pkl", 'rb') as f:
    playability9 = pkl.load(f)
with open("all_scores_nrows_10000_last_cat.pkl", 'rb') as f:
    playability10 = pkl.load(f)



In [ ]:
all_thingies = {0:[],1:[],2:[],3:[],4:[],5:[],6:[],7:[]}
for i in range(8):
    all_thingies[i] += playability0[i] +playability1[i] +playability2[i] +playability3[i] +playability4[i] +playability5[i] +playability6[i] +playability7[i] +playability8[i] +playability9[i] + playability10[i]
    
    

In [ ]:
with open("dotsplit_all_scores_nrows_10000.pkl", 'rb') as f:
    dotsplit_playability0 = pkl.load(f)
with open("dotsplit_all_scores_10000_20000.pkl", 'rb') as f:
    dotsplit_playability1 = pkl.load(f)
with open("dotsplit_all_scores_20000_30000.pkl", 'rb') as f:
    dotsplit_playability2 = pkl.load(f)
with open("dotsplit_all_scores_30000_40000.pkl", 'rb') as f:
    dotsplit_playability3 = pkl.load(f)
with open("dotsplit_all_scores_40000_50000.pkl", 'rb') as f:
    dotsplit_playability4 = pkl.load(f)
with open("dotsplit_all_scores_50000_60000.pkl", 'rb') as f:
    dotsplit_playability5 = pkl.load(f)
with open("dotsplit_all_scores_60000_70000.pkl", 'rb') as f:
    dotsplit_playability6 = pkl.load(f)
with open("dotsplit_all_scores_70000_80000.pkl", 'rb') as f:
    dotsplit_playability7 = pkl.load(f)
with open("dotsplit_all_scores_80000_90000.pkl", 'rb') as f:
    dotsplit_playability8 = pkl.load(f)
with open("dotsplit_all_scores_90000_100000.pkl", 'rb') as f:
    dotsplit_playability9 = pkl.load(f)
    
dotsplit_all_thingies = {0:[],1:[],2:[],3:[],4:[],5:[],6:[],7:[]}
for i in range(8):
    dotsplit_all_thingies[i] += dotsplit_playability0[i] + dotsplit_playability1[i] + dotsplit_playability2[i] + dotsplit_playability3[i] + dotsplit_playability4[i] + dotsplit_playability5[i] + dotsplit_playability6[i] + dotsplit_playability7[i] + dotsplit_playability8[i] + dotsplit_playability9[i]
    


In [ ]:
for i in range(8):
    dotsplit_all_thingies[i] = np.asarray(dotsplit_all_thingies[i]) /3 if i != 7 else np.asarray(dotsplit_all_thingies[i]) /39
    dotsplit_all_thingies[i] = dotsplit_all_thingies[i].round(4)
    print(np.asarray(dotsplit_all_thingies[i]).mean(), np.asarray(dotsplit_all_thingies[i]).var(), np.asarray(dotsplit_all_thingies[i]).min(), np.asarray(dotsplit_all_thingies[i]).max()) 
          

In [ ]:
dotsplit_all_thingies[i] = np.asarray(dotsplit_all_thingies[i]) /3 if i != 7 else np.asarray(dotsplit_all_thingies[i]) /39
dotsplit_all_thingies[i] = dotsplit_all_thingies[i].round(4)


In [ ]:
added_cols= ['GRU_char_CFP','GRU_char_CFD','GRU_char_UC','GRU_char_RHC','GRU_char_CPT','GRU_char_BD','GRU_char_R','GRU_char_weighted_total']

added_dotsplit_cols= ['GRU_dotsplit_CFP','GRU_dotsplit_CFD','GRU_dotsplit_UC','GRU_dotsplit_RHC','GRU_dotsplit_CPT','GRU_dotsplit_BD','GRU_dotsplit_R','GRU_dotsplit_weighted_total']


In [ ]:
# for i, col in enumerate(added_cols):
#     all_thingies[col] = all_thingies[i]
#     del all_thingies[i]
for i, col in enumerate(added_dotsplit_cols):
    dotsplit_all_thingies[col] = dotsplit_all_thingies[i]
    del dotsplit_all_thingies[i]

In [ ]:
for col in added_dotsplit_cols:
    print(col)
    custom_data_rule_based[col] = dotsplit_all_thingies[col]

In [ ]:
custom_data_rule_based.columns

In [ ]:
for col in added_cols:
    custom_data_rule_based[col] = all_thingies[col]

In [ ]:
custom_data_rule_based.to_csv("chords_with_transpose_difficulties_and_playability.csv")

In [ ]:
custom_data_rule_based[['difficulty', 'GRU_char_weighted_total','GRU_dotsplit_weighted_total','RB_weighted_total']]